In [1]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle

WORKING_DIRECTORY = '/media/dean/datastore/datasets/darknet_evaluate'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data', 'coco')
BDD10K_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'bdd10k_instances_train2014.json')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images','train2014')
WEIGHTS_DIRECTORY = os.path.join(WORKING_DIRECTORY,'trained_weights')

In [2]:
bdd10k_coco = COCO(BDD10K_COCO_ANNOTATIONS_FILE)
ann_ids = bdd10k_coco.getAnnIds()
print(len(ann_ids))

loading annotations into memory...
Done (t=1.21s)
creating index...
index created!
112110


In [3]:
# Grab batches and sort checkpoints
d = {}
weights_files = glob.glob(os.path.join(WEIGHTS_DIRECTORY, '*.weights'))

for fpath in weights_files:
    fname = os.path.split(fpath)[-1]
    batches = fname[15:].split('.weights')[0]
    d[fname] = int(batches)
sorted_weights = OrderedDict(sorted(d.items(), key=lambda kv: kv[1]))
bdd_weights = OrderedDict()

for weights_file in sorted_weights.keys():
    d = {}
    d['batches'] = sorted_weights[weights_file]
    d['map_results_file'] = os.path.join(WEIGHTS_DIRECTORY, weights_file+'.txt')
    map_results_file = d['map_results_file']
    
    if not os.path.exists(d['map_results_file']+'.backup'):
        # Create annotations if labels directory not found
        if not os.path.exists(os.path.join(COCO_DIRECTORY,'labels')):
            !java -jar $WORKING_DIRECTORY/data/cocotoyolo.jar "{BDD10K_COCO_ANNOTATIONS_FILE}" "{IMAGES_DIRECTORY}/" "all" "{COCO_DIRECTORY}/labels/train2014"
        
        outfile = !cd $WORKING_DIRECTORY && ./darknet detector map cfg/bdd100k.data cfg/yolov3-bdd100k.cfg $WEIGHTS_DIRECTORY/$weights_file | tee -a $map_results_file
        with open(d['map_results_file']+'.backup',"w+") as f:
            outfile = '\n'.join(outfile)
            f.write(outfile)
    
    #Get mAP results from file
    class_stats = []
    map_stats = {}
    with open(d['map_results_file']+'.backup') as openfile:
        for line in openfile:
            for part in line.split():
                if 'class_id' in part:
                    class_map = {}
                    id_tokens = line.split('class_id =')[1].split(',')
                    class_map['class_id'] = int(id_tokens[0].strip())

                    name_tokens = line.split('name =')[1].split(',')
                    class_map['class_name'] = name_tokens[0]

                    if 'ap =' in line:
                        ap_tokens = line.split('ap =')[1].split(',')
                        class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                    class_stats.append(class_map)
                elif '(mAP)' in part:
                    tokens = line.split('(mAP) =')[1]
                    map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
                elif 'Detection' in part:
                    tokens = line.split(':')[1]
                    total_detection_time = str(''.join(tokens)).strip()
                    map_stats['total_detection_time'] = total_detection_time
    map_stats['class_stats'] = class_stats
    print(map_stats,'\n\n') 
    bdd_weights[weights_file] = map_stats

    
    
# Make sure Dictionary is made properly
print(bdd_weights)

{'class_stats': []} 


{'total_detection_time': '332.000000 Seconds', 'mean_avg_precision': '2.86', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '0.36'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '0.01'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '13.58'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '0.02'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '0.00'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '4.55'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.00'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.56'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.00'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '9.51'}]} 


{'total_detection_time': '314.000000 Seconds', 'mean_avg_precision': '5.23', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '9.42'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '0.00'}, {'class_id': 2, 'class_name': ' car', 'cla

{'total_detection_time': '305.000000 Seconds', 'mean_avg_precision': '7.73', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '10.92'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.71'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '41.81'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '0.02'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '0.00'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '0.21'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.00'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.01'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.00'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '22.59'}]} 


{'total_detection_time': '297.000000 Seconds', 'mean_avg_precision': '9.71', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '11.39'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '9.27'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '46.12'}, {'

{'total_detection_time': '300.000000 Seconds', 'mean_avg_precision': '9.26', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '13.58'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.59'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '47.03'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '0.01'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '0.00'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '0.83'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.00'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.00'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.00'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '29.57'}]} 


{'total_detection_time': '296.000000 Seconds', 'mean_avg_precision': '9.74', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '15.13'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '2.26'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '47.68'}, {'

{'total_detection_time': '294.000000 Seconds', 'mean_avg_precision': '10.56', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '16.22'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.82'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '49.32'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '0.01'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '0.00'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '5.45'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.00'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.01'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.00'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '32.77'}]} 


{'total_detection_time': '299.000000 Seconds', 'mean_avg_precision': '10.13', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '15.91'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.65'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '46.23'}, 

{'total_detection_time': '295.000000 Seconds', 'mean_avg_precision': '10.81', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '19.08'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.13'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '49.27'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '0.01'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '0.00'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '9.09'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.00'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.00'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.00'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '29.52'}]} 


{'total_detection_time': '295.000000 Seconds', 'mean_avg_precision': '10.98', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '16.38'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.04'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '49.75'}, 

{'total_detection_time': '294.000000 Seconds', 'mean_avg_precision': '10.07', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '15.75'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.00'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '50.71'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '0.01'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '0.00'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '9.09'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.00'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.00'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.00'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '24.14'}]} 


{'total_detection_time': '291.000000 Seconds', 'mean_avg_precision': '11.26', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '17.09'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.64'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '50.17'}, 

{'total_detection_time': '292.000000 Seconds', 'mean_avg_precision': '11.02', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '17.66'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.21'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '48.19'}, {'class_id': 3, 'class_name': ' truck', 'class_ap': '0.01'}, {'class_id': 4, 'class_name': ' bus', 'class_ap': '0.00'}, {'class_id': 5, 'class_name': ' train', 'class_ap': '9.09'}, {'class_id': 6, 'class_name': ' motor', 'class_ap': '0.00'}, {'class_id': 7, 'class_name': ' bike', 'class_ap': '0.00'}, {'class_id': 8, 'class_name': ' traffic sign', 'class_ap': '0.00'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '34.02'}]} 


{'total_detection_time': '292.000000 Seconds', 'mean_avg_precision': '11.58', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '19.01'}, {'class_id': 1, 'class_name': ' rider', 'class_ap': '1.77'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '51.07'}, 

In [4]:
print(len(bdd_weights))

100
